In [5]:
import random
import functools

In [15]:
# Dictionary representing in-game locations
way_to_busto = {
    0: "the shire",
    1: "rivendell",
    2: "mines of moria",
    3: "lothlorien",
    4: "rohan",
    5: "helm's deep",
    6: "gondor",
    7: "minas tirith",
    8: "shelob's den",
    9: "mount doom"
}

class Die:
    def __init__(self, colour="indeterminate", current_face="indeterminate"):
        """
        Initialize a Die object with a specified color and current face.

        Parameters:
            colour (str): Color of the die.
            current_face (str): Current face of the die.
        """
        self.colour = colour
        self.current_face = current_face 

        if colour == "black":
            self.faces = ["ring", "nazgul", "nazgul", "orc", "tree", "fellowship"]
        else: 
            self.faces = ["ring", "nazgul", "gandalf", "orc", "fellowship", "fellowship"]

class Character:
    def __init__(self, name, colour, is_playing=True, is_player=False, current_position=0,
                 allowed_nazguls=6, taken_nazguls=0, gandalfs=0, allowed_gandalfs=6, played_turns=0):
        """
        Initialize a Character object with specified attributes.

        Parameters:
            name (str): Name of the character.
            colour (str): Color of the character.
            is_playing (bool): Whether the character is actively playing.
            is_player (bool): Whether the character is controlled by a player.
            current_position (int): Current position of the character on the game board.
            allowed_nazguls (int): Maximum number of nazguls the character can encounter.
            taken_nazguls (int): Number of nazguls already encountered.
            gandalfs (int): Number of gandalfs possessed by the character.
            allowed_gandalfs (int): Maximum number of gandalfs the character can use.
            played_turns (int): Number of turns the character has played.
        """
        self.is_playing = is_playing
        self.is_player = is_player
        self.name = name
        self.colour = colour
        self.current_position = current_position
        self.allowed_nazguls = allowed_nazguls
        self.taken_nazguls = taken_nazguls
        self.gandalfs = gandalfs
        self.allowed_gandalfs = allowed_gandalfs
        self.played_turns = played_turns

    def bot_dice(self):
        """
        Simulate the dice roll for a bot character (warning: very much wip).

        Returns:
            list: List of Die objects representing the dice rolled.
        """
        yellow_die = Die("yellow")
        red_die = Die("red")
        blue_die = Die("blue")
        green_die = Die("green")
        black_die = Die("black")
        
        dice = [yellow_die, red_die, blue_die, green_die, black_die]
        
        chosen_dice = []
        
        for die in dice: 
            die.current_face = random.choice(die.faces)
            
        nazguls = [die for die in dice if die.current_face == "nazgul"]
        nazgul_colours = [die.colour for die in nazguls]
        
        if len(nazguls) == 1:
            chosen_dice.append(*nazguls)
        
        # don't choose your own colour if you can!
        if len(nazguls) >= 2:
            if self.colour in nazgul_colours:
                other_nazguls = [nazgul for nazgul in nazguls if nazgul.colour != self.colour]
                chosen_dice.append(random.choice(other_nazguls))
            else:
                chosen_dice.append(random.choice(nazguls))
        
        return dice

    def player_dice(self):
        """
        Simulate the dice roll and player's choice for a player character.

        Returns:
            list: List of Die objects representing the dice chosen by the player.
        """
        yellow_die = Die("yellow")
        red_die = Die("red")
        blue_die = Die("blue")
        green_die = Die("green")
        black_die = Die("black")

        dice_remaining = [yellow_die, red_die, blue_die, green_die, black_die]
        dice_taken = []

        while dice_remaining:
            dice_on_table = []

            for die in dice_remaining:
                die.current_face = random.choice(die.faces)
                dice_on_table.append(die)

            print("New throw \n")

            current_faces = [die.current_face for die in dice_on_table]
            current_colours = [die.colour for die in dice_on_table]

            chosen_dice = []
            choice = Die()

            # mandatory nazgul pick
            current_nazguls = [die for die in dice_on_table if die.current_face == "nazgul"]
            current_nazguls_colours = [die.colour for die in current_nazguls]
            
            if current_nazguls:
                print("NAZGUL PHASE \n")
                print("There are nazguls of colour: ")
                    
                for nazgul in current_nazguls:
                    print(f"{nazgul.colour} ")
                    
                print(".\n")

                while not choice.colour in current_nazguls_colours:
                    choice.colour = input("You must choose one (write its colour): ")
                    print("\n")
                    
                chosen_nazgul = [die for die in current_nazguls if die.colour == choice.colour]
                for nazgul in chosen_nazgul:
                    print(f"You have picked the {nazgul.colour} nazgul. \n")

                chosen_dice.append(*chosen_nazgul)
                dice_on_table.remove(*chosen_nazgul)

            # player's choice
            print("Free choice\n")
            
            current_faces = [die.current_face for die in dice_on_table]
            current_colours = [die.colour for die in dice_on_table]
            chosen_faces = [die.current_face for die in chosen_dice]

            while dice_on_table:
                choice = Die()

                print(f"These are the dice on the table: { {current_colours[i] : current_faces[i] for i in range(len(current_colours))} } \n")
                print(f"These are the chosen types: {chosen_faces}\n")

                while(choice.current_face == "indeterminate"):
                    print("start loop")

                    choice.colour = input("Choose a die (write its colour) or write \"stop\" to finish round and throw remaining dice again: ")
                    print("\n")

                    if choice.colour == "stop" and chosen_dice:
                        break
                    
                    elif choice.colour == "stop" and not chosen_dice:
                        print("You must choose at least one die.")
                        choice.current_face == "indeterminate"
                    
                    elif not choice.colour in current_colours:
                        print(f"You must pick one of these colours: {current_colours}. \n")
                        choice.current_face == "indeterminate"
                    
                    else:
                        print("entering colour case")
                        for die in dice_on_table:
                            if die.colour == choice.colour:
                                choice.current_face = die.current_face

                        if choice.current_face in chosen_faces:
                            print("entering multiple faces case")
                            choice.current_face = "indeterminate"

                    print(f"exiting inside loop with value {choice.current_face}")

                print("outside loop")

                if choice.colour == "stop":
                    break

                chosen_dice.append(*[die for die in dice_on_table if die.colour == choice.colour])
                dice_on_table.remove(*[die for die in dice_on_table if die.colour == choice.colour])

                current_faces = [die.current_face for die in dice_on_table]
                current_colours = [die.colour for die in dice_on_table]
                chosen_faces = [die.current_face for die in chosen_dice]

            for die in chosen_dice:
                dice_remaining.remove(die)
                dice_taken.append(die)

            if "tree" in chosen_faces:
                print("Turn ends because white tree has been chosen. \n")
                break

            chosen_dice = []

            if "tree" in chosen_faces:
                print("Turn ends because white tree has been chosen. \n")
                break

        return dice_taken

    def apply_dice(self, opponents, dice_taken):
        """
        Apply the effects of the chosen dice to the character and opponents.

        Parameters:
            opponents (list): List of Character objects representing opponents.
            dice_taken (list): List of Die objects representing the dice chosen by the player.

        Returns:
            tuple: A tuple containing a list of surviving characters and a list of winners (if any).
        """
        self.played_turns += 1
        faces = [die.current_face for die in dice_taken]

        # determine movement
        n_orcs = faces.count("orc")
        n_fellowships = faces.count("fellowship")

        can_move = n_fellowships - n_orcs > 0

        if can_move:
            n_rings = faces.count("ring")
            self.current_position += n_rings

        # determine gandalfs
        n_gandalfs = 0

        for face in faces:
            if face == "gandalf" and n_gandalfs < self.allowed_gandalfs:
                n_gandalfs += 1
            elif n_gandalfs == self.allowed_gandalfs:
                break

        self.gandalfs += n_gandalfs
        self.allowed_nazguls += n_gandalfs

        # determine nazguls
        nazguls = [die for die in dice_taken if die.current_face == "nazgul"]

        for nazgul in nazguls:
            colour = nazgul.colour

            if colour == self.colour:
                self.taken_nazguls += 1

            for hobbit in opponents:
                if hobbit.colour == colour:
                    hobbit.taken_nazguls += 1

        # handle losses and wins
        survivors = [self, *opponents]
        winners = []  # there could be multiple winners

        for hobbit in survivors:
            if hobbit.taken_nazguls == hobbit.allowed_nazguls:
                print(f"{hobbit.name} has lost the game!")
                survivors.remove(hobbit)

            if hobbit.current_position == 9 and hobbit.taken_nazguls < hobbit.allowed_nazguls:
                winners.append(hobbit)

        return survivors, winners

In [23]:
Frodo = Character(name = "Frodo", colour = "yellow")
Sam = Character(name = "Sam", colour = "red")
Merry = Character(name = "Merry", colour = "blue")
Pippin = Character(name = "Pippin", colour = "green")

characters = [Frodo, Sam, Merry, Pippin]

character_names = [character.name for character in characters]
player_name = None

while not player_name in character_names:
    
    player_name = input("\n\
> Choose your character.\n\n\
    [Frodo]        [Sam]\n\
    [Merry]        [Pippin]\n")
    
for character in characters:
    if character.name == player_name: character.is_player = True
        
winners = []
past_losers = []
    
while not winners:
    
    # one round is a turn for everybody
    for character in characters: 
        
        # update characters to alive hobbits
        characters = [hobbit for hobbit in characters if not hobbit in past_losers]
        # check if current character has lost, if so skip 
        if not character in characters:
            print(f"{character.name} is dead, so he does nothing \n")
            pass
        
        else:
    
            print(f"These are the characters in the game: {[character.name for character in characters]}")
            print(f"It's {character.name} turn.")

            # determine opponents of currently playing character
            opponents = [hobbit for hobbit in characters if hobbit != character]
            
            opponents_names = [hobbit.name for hobbit in opponents]
            print(f"These are his opponents: {opponents_names}")

            # if current character is the player call the appropriate function, else randomize
            if character.is_player:
                dice = character.bot_dice() # dice = character.player_dice()

            else:
                dice = character.bot_dice()

            faces = [die.current_face for die in dice]
            print(f"{character.name} has thrown {faces}. \n")

            # evaluate hand and determine losers and winner
            survivors, winner = character.apply_dice(opponents, dice)
            losers = [hobbit for hobbit in characters if not hobbit in survivors]
            
            if losers:
                past_losers.append(*losers) # keeps track of all hobbits that have lost

            if winner:
                winners.append(winner) # note: there can be one winner per turn but multiple losers

winner_names = [winner[0].name for winner in winners]
print(f"The winners are {list(set(winner_names))}.")


> Choose your character.

    [Frodo]        [Sam]
    [Merry]        [Pippin]
Merry
These are the characters in the game: ['Frodo', 'Sam', 'Merry', 'Pippin']
It's Frodo turn.
These are his opponents: ['Sam', 'Merry', 'Pippin']
Frodo has thrown ['gandalf', 'gandalf', 'ring', 'gandalf', 'nazgul']. 

These are the characters in the game: ['Frodo', 'Sam', 'Merry', 'Pippin']
It's Sam turn.
These are his opponents: ['Frodo', 'Merry', 'Pippin']
Sam has thrown ['gandalf', 'nazgul', 'gandalf', 'gandalf', 'fellowship']. 

These are the characters in the game: ['Frodo', 'Sam', 'Merry', 'Pippin']
It's Merry turn.
These are his opponents: ['Frodo', 'Sam', 'Pippin']
Merry has thrown ['gandalf', 'ring', 'fellowship', 'fellowship', 'tree']. 

These are the characters in the game: ['Frodo', 'Sam', 'Merry', 'Pippin']
It's Pippin turn.
These are his opponents: ['Frodo', 'Sam', 'Merry']
Pippin has thrown ['nazgul', 'orc', 'nazgul', 'ring', 'ring']. 

These are the characters in the game: ['Frodo', 'Sam'